# Lab-spotify-api


In [1]:
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
import requests

#### previous lab

In [2]:
nothot100 = pd.read_csv("Spotify_final_dataset.csv")

In [3]:
nothot100.head()

,Position,Artist Name,Song Name,Days,Top 10 (xTimes),Peak Position,Peak Position (xTimes),Peak Streams,Total Streams
0,1,Post Malone,Sunflower SpiderMan: Into the SpiderVerse,1506,302.0,1,(x29),2118242,883369738
1,2,Juice WRLD,Lucid Dreams,1673,178.0,1,(x20),2127668,864832399
2,3,Lil Uzi Vert,XO TOUR Llif3,1853,212.0,1,(x4),1660502,781153024
3,4,J. Cole,No Role Modelz,2547,6.0,7,0,659366,734857487
4,5,Post Malone,rockstar,1223,186.0,1,(x124),2905678,718865961


In [4]:
nothot100 = nothot100.sample(2500)

In [5]:
nothot100 = nothot100[["Artist Name","Song Name"]]

In [6]:
nothot100 = nothot100.rename(columns={'Artist Name': 'Artist', 'Song Name': 'Title'})

In [7]:
hot100 = pd.read_csv("hot100.csv")

In [8]:
concatenated_df = pd.concat([hot100, nothot100]).drop_duplicates()

In [9]:
concatenated_df = concatenated_df.reset_index(drop=True)

In [10]:
concatenated_df

,Artist,Title
0,Olivia Rodrigo,Vampire
1,Morgan Wallen,Last Night
2,Luke Combs,Fast Car
3,Rema & Selena Gomez,Calm Down
4,Miley Cyrus,Flowers
...,...,...
2589,Chucho Flash,Tu Sabes Que Te Quiero
2590,One Direction,You & I
2591,Lizzo,Soulmate
2592,Lil Uzi Vert,Secure The Bag


In [11]:
data = concatenated_df.copy()

### Lab Spotify api

In [12]:
from credencials import *

In [13]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret=client_secret))

In [14]:
data1 = data.head(1)
data1

,Artist,Title
0,Olivia Rodrigo,Vampire


In [15]:
def search_song(title, artist):
    
    query = "tracks: "+title+" artist: "+artist
    #print(query)

    try:
        result = sp.search(query, limit=1)
        song_id = result['tracks']['items'][0]['id']
    except:
        song_id = "not_found"

    return song_id  

In [16]:
search_song("Hello", "Adele")

'1AynHbA5tcynFDplpCY5lz'

In [17]:
df1 = hot100
df2 = nothot100
df2.isnull().sum()

Artist    0
Title     0
dtype: int64

In [18]:
len(df1)

100

In [19]:
df2 = df2.dropna()
len(df2)

2500

In [20]:
# create groups fo 50 song for df1
chunks_df1 = [df1.iloc[i:i+50] for i in range(0, len(df1), 50)]

In [21]:
chunks_df2 = [df2[i:i + 50] for i in range(0, len(df2), 50)]


#### df1

In [22]:
#create the list
import time
song_ids_df1 = []
for chunk in chunks_df1:
    for index, row in chunk.iterrows():
        song_id = search_song(row["Title"], row["Artist"])
        if song_id == 'not_found':
            print("yes")
        song_ids_df1.append(song_id)
    time.sleep(20)
print('finished')

finished


In [23]:
len(song_ids_df1)

100

In [24]:
## apply get_audio_features
def get_audio_features(list_of_song_ids):
    
    chunks = [list_of_song_ids[i:i + 50] for i in range(0, len(list_of_song_ids), 50)]
    
    # Create a dictionary to store the audio features
    audio_features_dict = {
        'id': [],
        'danceability': [],
        'energy': [],
        'key': [],
        'loudness': [],
        'mode': [],
        'speechiness': [],
        'acousticness': [],
        'instrumentalness': [],
        'liveness': [],
        'valence': [],
        'tempo': [],
        'type': [],
        'uri': [],
        'track_href': [],
        'analysis_url': [],
        'duration_ms': [],
        'time_signature': []
    }
    
    # Retrieve the audio features for each chunk
    for chunk in chunks:
        audio_features = sp.audio_features(chunk)
        
        # Append the audio features to the dictionary
        for feature in audio_features:
            if feature:
                for key in audio_features_dict.keys():
                    audio_features_dict[key].append(feature[key])
        
        # Wait 20 seconds before next API call
        time.sleep(20)

    # Create a DataFrame from the dictionary
    df = pd.DataFrame(audio_features_dict)

    return df

In [25]:
df3 = get_audio_features(song_ids_df1)
len(df3)

100

In [26]:
result = pd.concat([df1, df3], axis=1)
result.head()

,Artist,Title,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Olivia Rodrigo,Vampire,4sSKTQcX6pQIncWcNJdkiD,0.675,0.609,1,-7.905,1,0.0940,0.2840,0.268000,0.112,0.057,119.973,audio_features,spotify:track:4sSKTQcX6pQIncWcNJdkiD,https://api.spotify.com/v1/tracks/4sSKTQcX6pQI...,https://api.spotify.com/v1/audio-analysis/4sSK...,300000,4
1,Morgan Wallen,Last Night,59uQI0PADDKeE6UZDTJEe8,0.517,0.675,6,-5.382,1,0.0357,0.4590,0.000000,0.151,0.518,203.853,audio_features,spotify:track:59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://api.spotify.com/v1/audio-analysis/59uQ...,163855,4
2,Luke Combs,Fast Car,2M9ro2krNb7nr7HSprkEgo,0.711,0.292,4,-15.523,0,0.0370,0.3130,0.000000,0.131,0.194,103.951,audio_features,spotify:track:2M9ro2krNb7nr7HSprkEgo,https://api.spotify.com/v1/tracks/2M9ro2krNb7n...,https://api.spotify.com/v1/audio-analysis/2M9r...,296800,4
3,Rema & Selena Gomez,Calm Down,0WtM2NBVQNNJLh6scP13H8,0.801,0.806,11,-5.206,1,0.0381,0.3820,0.000669,0.114,0.802,106.999,audio_features,spotify:track:0WtM2NBVQNNJLh6scP13H8,https://api.spotify.com/v1/tracks/0WtM2NBVQNNJ...,https://api.spotify.com/v1/audio-analysis/0WtM...,239318,4
4,Miley Cyrus,Flowers,6ugrRFZUNIpLiqhLUgC7ix,0.838,0.364,7,-10.006,1,0.1340,0.0849,0.981000,0.260,0.218,79.987,audio_features,spotify:track:6ugrRFZUNIpLiqhLUgC7ix,https://api.spotify.com/v1/tracks/6ugrRFZUNIpL...,https://api.spotify.com/v1/audio-analysis/6ugr...,187828,4


In [27]:
result.shape

(100, 20)

In [28]:
# create the function accoding to the result
def add_audio_features(df: pd.DataFrame, audio_features_df: pd.DataFrame):
    '''
    This fucntion concatanates two dataframes.
    Input pd.Dataframe
    Output None
    '''
    result = pd.concat([df, audio_features_df], axis=1)
    name = 'merged_songs' + str(len(result)) + '.csv'
    result.to_csv(name, index=False)
    

In [29]:
add_audio_features(df1, df3)

### df2

In [30]:
#df2
import time
song_ids_df2 = []
for chunk in chunks_df2:
    for index, row in chunk.iterrows():
        song_id = search_song(row["Title"], row["Artist"])
        if song_id == 'not_found':
            print("yes")
        song_ids_df2.append(song_id)
    time.sleep(20)
print('finished')

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
finished


In [31]:
len(song_ids_df2)

2500

In [55]:
#to get the index which has 'not_found'
index_list = []
for i in range(0,2500):
    if song_ids_df2[i] == 'not_found':
        index_list.append(i)
index_list    

[73, 420, 596, 843, 877, 1109, 1196, 1509, 1762, 2034, 2091, 2371]

In [33]:
song_ids_df3 = song_ids_df2.copy()
#remove 'not_found' values 
song_ids_df3 = [x for x in song_ids_df3 if x != 'not_found']
len(song_ids_df3)

2488

In [34]:
# remove specific rows based on the given indices
df8 = df2.copy()
df9 = df8.reset_index(drop=True)
df10 = df9.drop(index_list)
len(df10)


2488

In [35]:
df11 = get_audio_features(song_ids_df3)

In [37]:
len(df11)

2486

In [46]:
# some Ids are missed. So, I search the value. 
for i in song_ids_df3:
    if i not in df11['id'].valslues:
        print(i)

6yqkj5RfE5vOHf4vwCt6m0
43RQWXKpp5mSWt0i2DGOTx


In [49]:
#get the index which are not included in df11 but in song_ids_df3 
index_list2 = []
for i in range(0,2488):
    if (song_ids_df3[i] == '6yqkj5RfE5vOHf4vwCt6m0') or (song_ids_df3[i] == '43RQWXKpp5mSWt0i2DGOTx'):
        index_list2.append(i)
index_list2

[592, 1710]

In [51]:
## remove the missed values 

df12 = df10.drop(index_list2)
len(df12)

2486

In [52]:
df13 = df12.reset_index(drop=True)
df13

,Artist,Title
0,Ed Sheeran,The A Team
1,Fifth Harmony,BO$$
2,Rod Wave,Pray 4 Love
3,Mac DeMarco,Chamber Of Reflection
4,Christina Aguilera,Have Yourself a Merry Little Christmas
...,...,...
2481,Chucho Flash,Tu Sabes Que Te Quiero
2482,One Direction,You & I
2483,Lizzo,Soulmate
2484,Lil Uzi Vert,Secure The Bag


In [54]:
#df13 is old data and df11 is the expanded data.
add_audio_features(df13, df11)